In [1]:
import os
import csv
import pandas as pd
import numpy as np
from matplotlib import pyplot
import math

In [23]:

df = pd.DataFrame()
header_first = True
mode='w'
for root, dirs, files in os.walk('results'):
    for f in files: 
        if(f.startswith('Result_')):
          file_name=os.path.join(root, f)
          model = f.split('_')[1]
          frequence = f.split('_')[3].split('.')[0]
          dataset = pd.read_csv(file_name, sep =';', encoding = 'latin1', decimal='.')
          dataset['model']=model
          dataset['freq']=frequence
          dataset['freq'].replace(['Hour','Day','Week'],['1-Hour','2-Day','3-Week'],inplace=True)
          dataset.columns=[ 'SK_PONTO','Best_Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration','model','freq']
          dataset.to_csv('tables/all_results.csv',sep=';',mode=mode,index=False, header=header_first)
          header_first = False
          mode='a'
        


In [24]:
dataset = pd.read_csv(f'results/all_results.csv', sep =';', encoding = 'latin1', decimal='.')
dataset['MAE']=round(dataset['MAE'].str.replace(',','.').astype(float),2)
dataset['MSE']=round(dataset['MSE'].str.replace(',','.').astype(float),2)
dataset['RMSE']=round(dataset['RMSE'].str.replace(',','.').astype(float),2)
dataset['MAPE']=round(dataset['MAPE'].str.replace(',','.').astype(float),2)
dataset['Duration']=round(dataset['Duration'].str.replace(',','.').astype(float),1)

fs=dataset.pop('freq')
dataset.insert(1, 'freq', fs)

fs=dataset.pop('model')
dataset.insert(3, 'model', fs)

fs=dataset.pop('Best_Params')
dataset.insert(9, 'Best_Params', fs)

idx = dataset.groupby(['SK_PONTO','freq'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].sort_values(['SK_PONTO','freq']).to_csv('tables/best_model_per_reservior.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="2-Day"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_model_per_reservior_day.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="1-Hour"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_model_per_reservior_hour.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="3-Week"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_model_per_reservior_week.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="2-Day"').sort_values(['SK_PONTO','freq']).head()

,SK_PONTO,freq,model,N_Past_Vl,MSE,RMSE,MAE,MAPE,Duration,Best_Params
183,1,2-Day,ARIMA2,4,3972.49,63.03,43.61,6.64,146.6,"ARIMA(3,1,0)(0,0,0)"
326,4,2-Day,LSTM,2,3960186.53,1990.02,330.86,8.64,8.8,"LSTM(100,96,90)"
333,5,2-Day,LSTM,3,3524767.56,1877.44,669.97,8.35,9.1,"LSTM(100,96,90)"
520,6,2-Day,MLP,4,260528.70,510.42,195.76,5.46,38.6,"{'activation': 'identity', 'alpha': 0,0001, 'h..."
204,7,2-Day,ARIMA2,1,2207.54,46.98,36.44,6.45,31.1,"ARIMA(0,1,1)(0,0,0)"


In [25]:
idx = dataset.groupby(['SK_PONTO','freq','model'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].query('model=="SVR-LSTM" and freq=="1-Hour"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrsltm_best_model_per_reservior_hour.csv',sep=';',decimal=',', index=False)
dataset[idx].query('model=="SVR-LSTM" and freq=="2-Day"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrsltm_best_model_per_reservior_day.csv',sep=';',decimal=',', index=False)
dataset[idx].query('model=="SVR-LSTM" and freq=="3-Week"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrsltm_best_model_per_reservior_week.csv',sep=';',decimal=',', index=False)


In [26]:
idx = dataset.groupby(['SK_PONTO','freq','model'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].query('model=="SVR-MLP" and freq=="1-Hour"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrmlp_best_model_per_reservior_hour.csv',sep=';',decimal=',', index=False)
dataset[idx].query('model=="SVR-MLP" and freq=="2-Day"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrmlp_best_model_per_reservior_day.csv',sep=';',decimal=',', index=False)
dataset[idx].query('model=="SVR-MLP" and freq=="3-Week"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrmlp_best_model_per_reservior_week.csv',sep=';',decimal=',', index=False)

In [27]:
dataset['SVR-LSTM']=np.where(dataset['model']=='SVR-LSTM',dataset['MAPE'],999)
dataset['SVR-MLP']=np.where(dataset['model']=='SVR-MLP',dataset['MAPE'],999)
dataset['SVR']=np.where(dataset['model']=='SVR',dataset['MAPE'],999)
dataset['LSTM']=np.where(dataset['model']=='LSTM',dataset['MAPE'],999)
dataset['ARIMA1']=np.where(dataset['model']=='ARIMA1',dataset['MAPE'],999)
dataset['ARIMA2']=np.where(dataset['model']=='ARIMA2',dataset['MAPE'],999)
dataset['MLP']=np.where(dataset['model']=='MLP',dataset['MAPE'],999)

In [28]:
idx = dataset.groupby(['SK_PONTO','freq','SVR-LSTM','SVR-MLP','SVR','MLP','LSTM','ARIMA1','ARIMA2'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].query('freq=="1-Hour"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_mape_per_reservior_hour.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="2-Day"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_mape_per_reservior_day.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="3-Week"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_mape_per_reservior_week.csv',sep=';',decimal=',', index=False)

In [36]:
dataset2=dataset.query('freq=="1-Hour"').groupby(['SK_PONTO','freq']).min(['SVR','LSTM','ARIMA1','ARIMA2','MLP','SVR-LSTM','SVR-MLP'])
dataset2

,,N_Past_Vl,MSE,RMSE,MAE,MAPE,Duration,SVR-LSTM,SVR-MLP,SVR,LSTM,ARIMA1,ARIMA2,MLP
SK_PONTO,freq,,,,,,,,,,,,,
1,1-Hour,1,1613.21,40.16,5.19,2.31,1.2,3.54,3.32,28.94,2.31,6.69,6.64,3.25
4,1-Hour,1,2186.08,46.76,18.09,4.93,2.0,10.13,11.96,15.70,4.93,11.88,11.85,9.02
5,1-Hour,1,3923.18,62.64,29.23,4.85,2.4,7.17,5.82,9.75,4.85,17.89,20.87,6.03
6,1-Hour,1,4175.91,64.62,28.15,8.00,2.3,34.41,34.32,8.00,84.53,38.02,52.47,34.69
7,1-Hour,1,1193.79,34.55,15.56,3.95,2.5,6.09,4.70,16.46,3.95,6.51,6.45,5.04
8,1-Hour,1,21.10,4.59,1.43,1.20,1.9,2.27,1.33,3.28,1.20,7.02,3.34,1.35
9,1-Hour,1,50.90,7.13,2.81,0.94,1.2,2.04,0.95,4.39,0.94,1.05,8.52,0.95
11,1-Hour,1,426.60,20.65,7.70,1.51,2.8,1.82,1.54,6.86,1.53,1.63,3.24,1.51
12,1-Hour,1,1216.21,34.87,15.68,6.33,2.8,9.87,8.09,6.33,7.34,12.61,11.71,8.01


In [22]:
dataset['SVR-LSTM']=np.where(dataset['model']=='SVR-LSTM',dataset['MAPE'],999)
dataset['SVR-MLP']=np.where(dataset['model']=='SVR-MLP',dataset['MAPE'],999)
dataset['SVR']=np.where(dataset['model']=='SVR',dataset['MAPE'],999)
dataset['LSTM']=np.where(dataset['model']=='LSTM',dataset['MAPE'],999)
dataset['ARIMA1']=np.where(dataset['model']=='ARIMA1',dataset['MAPE'],999)
dataset['ARIMA2']=np.where(dataset['model']=='ARIMA2',dataset['MAPE'],999)
dataset['MLP']=np.where(dataset['model']=='MLP',dataset['MAPE'],999)
dataset2=dataset.query('freq=="1-Hour"').groupby(['SK_PONTO','freq']).min(['SVR','LSTM','ARIMA1','ARIMA2','MLP','SVR-LSTM','SVR-MLP'])
dataset2

N_Past_Vl           MSE       RMSE       MAE   MAPE  \
SK_PONTO freq                                                          
1        1-Hour          1  1.613210e+03      40.16      5.19   2.31   
         2-Day           1  3.959880e+03      62.93     43.61   6.64   
         3-Week          1  2.580774e+08   16064.79   2687.51  14.16   
4        1-Hour          1  2.186080e+03      46.76     18.09   4.93   
         2-Day           1  7.605590e+03      87.21     65.35   8.64   
         3-Week          1  1.754973e+08   13247.54   1950.29  10.09   
5        1-Hour          1  3.923180e+03      62.64     29.23   4.85   
         2-Day           1  3.111940e+04     176.41    140.52   8.35   
         3-Week          1  2.815575e+08   16779.68   2228.65   5.25   
6        1-Hour          1  4.175910e+03      64.62     28.15   8.00   
         2-Day           1  1.124425e+04     106.04    100.31   5.46   
         3-Week          1  2.239233e+07    4732.05    569.75   3.39   
7        1-Hour          1  1.193790e+03      34.55     15.56   3.95   
         2-Day           1  2.207540e+03      46.98     36.44   6.45   
         3-Week          1  9.733083e+07    9865.64   3260.69   6.75   
8        1-Hour          1  2.110000e+01       4.59      1.43   1.20   
         2-Day           1  6.438000e+01       8.02      6.24   2.71   
         3-Week          1  1.147735e+07    3387.82    452.69   4.83   
9        1-Hour          1  5.090000e+01       7.13      2.81   0.94   
         2-Day           1  6.844000e+01       8.27      4.57   1.05   
         3-Week          1  1.342440e+08   11586.37   2270.71  22.15   
11       1-Hour          1  4.266000e+02      20.65      7.70   1.51   
         2-Day           1  4.414400e+02      21.01     13.80   1.63   
         3-Week          1  2.355965e+08   15349.15   2802.77   4.84   
12       1-Hour          1  1.216210e+03      34.87     15.68   6.33   
         2-Day           1  2.030920e+03      45.07     37.71   5.96   
         3-Week          1  3.187110e+07    5645.45   1046.48   4.31   
13       1-Hour          1  3.279977e+04     181.11     63.39   3.34   
         2-Day           1  1.147784e+05     338.79    262.37   7.44   
         3-Week          1  1.303095e+10  114153.20  20446.86  20.22   

                 Duration  SVR-LSTM  SVR-MLP    SVR   LSTM  ARIMA1  ARIMA2  \
SK_PONTO freq                                                                
1        1-Hour       1.2    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         2-Day        6.7    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         3-Week       0.1    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
4        1-Hour       2.0    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         2-Day        8.2    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         3-Week       0.2    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
5        1-Hour       2.4    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         2-Day        9.1    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         3-Week       0.2    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
6        1-Hour       2.3    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         2-Day        9.4    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         3-Week       0.2    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
7        1-Hour       2.5    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         2-Day        9.6    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         3-Week       0.2    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
8        1-Hour       1.9    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         2-Day        8.6    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         3-Week       0.1    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
9        1-Hour       1.2    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0   
         2-Day        6.4    -999.0   -999.0 -999.0 -999.0  -999.0  -999.0

In [25]:
idx=dataset.groupby(['SK_PONTO','model','freq','MAPE'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].sort_values(['SK_PONTO','freq']).to_csv('results/best_model_per_reservior.csv',sep=';',decimal=',', index=False)